## Create Dask Execution Cluster

In [1]:
import dask, os
from dask.distributed import Client, LocalCluster
import dask, dask.distributed
import numpy as np

In [2]:
cluster_type = 'local'
#cluster_type = 'HPC'

In [3]:
if cluster_type == 'local':
    dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})
    cluster = LocalCluster(n_workers=7,threads_per_worker=2)
    cl = Client(cluster)
elif cluster_type == 'HPC':
    import dask_jobqueue as jq
    dask.config.set({'distributed.dashboard.link': '/user/{USER}/proxy/{port}/status'})
    partition='brief-low'#,debug,mem,mem-low'
    num_processes = 4
    num_threads_per_processes = 8
    mem = 3.2*num_processes*num_threads_per_processes#*1.25
    n_cores_per_job = num_processes*num_threads_per_processes
    container = 'docker://rowangaffney/data_science_im_rs:latest'
    env = 'py_geo'
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            python="singularity -vv exec {} /opt/conda/envs/{}/bin/python".format(container,env),
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    cl=Client(clust)
    
    #Scale Cluster 
    num_jobs=8
    clust.scale(n=num_jobs*num_processes)
    cl.wait_for_workers(n_workers=num_jobs*num_processes)
else:
    print('Cluster type not defined')
ncpus = int(np.mean(np.array(list(cl.nthreads().values()))).round(0))
cl

2022-04-29 08:33:12,423 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/aop_mosaic/dask-worker-space/worker-1p3hey3z', purging
2022-04-29 08:33:12,433 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/aop_mosaic/dask-worker-space/worker-6hiv8kip', purging
2022-04-29 08:33:12,441 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/aop_mosaic/dask-worker-space/worker-9kxs1bx2', purging
2022-04-29 08:33:12,449 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/aop_mosaic/dask-worker-space/worker-a2ulldps', purging
2022-04-29 08:33:12,460 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/aop_mosaic/dask-worker-space/worker-hqg0f069', purging
2022-04-29 08:33:12,469 - distributed.diskutils - INFO - Found stale lock file a

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 15.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46203,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 15.50 GiB
Comm: tcp://127.0.0.1:39153,Total threads: 2
Dashboard: /proxy/41381/status,Memory: 2.21 GiB
Nanny: tcp://127.0.0.1:33511,


## Build Flow

In [4]:
#Import custom tasks
from aop_mosaic import tasks as t

In [5]:
#### Define Parameters ###
site_p = 'CPER'
processDate_p = '2017-05'
result_folder = default = './results'

if not os.path.exists(result_folder):
    os.mkdir(result_folder)


#Task1: Get metadata about the data product for a specific site / date
site_dict = t.query_data_urls(site=site_p,
                              processDate=processDate_p)

#Task2: Get the URLS for all the h5 files.
#pipeline_dict = {}
h5_files = t.query_file_urls(site_dict=site_dict,
                             site=site_p,
                             processDate=processDate_p,
                            result_folder=result_folder)
#pipeline_dict['images'] = h5_files

In [6]:
h5_files = [i for i in h5_files if i['name'] in os.listdir(result_folder + '/' + site_p + '_' + processDate_p)]


In [7]:
#Task3: Setup the BRDF and TOPO correction configurations
config_pipeline = []
for f in h5_files:
    config_pipeline.append(t.BRDF_TOPO_Config(pipeline_dict = f,
                                   site=site_p,
                                   processDate=processDate_p,
                                   cpus=ncpus))
#config_meta = dask.compute(*config_pipeline)

In [8]:
#Task4: Download the files to folder ./{site}_{procossDate}/
download_pipeline = []
for d in config_pipeline:
    download_pipeline.append(t.download_file(pipeline_dict = d,
                               site=site_p,
                               processDate=processDate_p,
                               result_folder=result_folder))
#download_meta = dask.compute(*download_pipeline)

In [9]:
#Task5: Get the metadata for each file
workflow_pipeline = []
for d in download_pipeline:
    workflow_pipeline.append(t.get_file_meta(pipeline_dict=d))
#workflow_meta = dask.compute(*workflow_pipeline)

In [10]:
#Task6: Write the metadata for each file to a human readable file (.json)
metadata_exported = t.write_pipeline_meta(pipeline_dict = workflow_pipeline,
                                          site=site_p,
                                          processDate=processDate_p,
                                          result_folder = result_folder)

In [11]:
#Task7: Apply the BRDF and TOPO corrections to the data
ht_pipeline1 = []
for d in workflow_pipeline:
    ht_pipeline1.append(t.apply_corrections_mosaic(pipeline_dict=d,
                                                      site=site_p,
                                                      processDate=processDate_p))
#ht_dict1 = dask.compute(*ht_pipeline1)

#Task8: Get the mask for each flight line for mosaicing the flights together
ht_pipeline2 = []
for d in ht_pipeline1:
    ht_pipeline2.append(t.pixel_mosaic_mask(pipeline_dict = d,
                                            pipeline_list=ht_pipeline1))

#Task9: Get the extents of all the flights for the final mosaic.
extent = t.moasic_extent(ht_pipeline2)              


In [12]:
#Task10: Mosaic the BRDF and Topo corrected flights using pixels to the lowest sensor to-zenith angle
success = t.mosaic(pipeline_list=ht_pipeline2,
                   extents=extent,
                   site=site_p,
                   processDate=processDate_p,
                   result_folder = result_folder)

In [ ]:
success.compute()
metadata_exported.compute()

{}
Skipping file: already downloaded!
Calculating topographic coefficients.
Scene average solar zenith angle : 21.622 degrees
Calculating BRDF coefficients
 ####################-------------------------------------------------------------------------------- [ 19.78%]{}
Skipping file: already downloaded!
Calculating topographic coefficients.
Scene average solar zenith angle : 42.911 degrees
Calculating BRDF coefficients
 ############################################################---------------------------------------- [ 60.45%]####################-------------------------------------------------------------------------------- [ 20.06%]####################-------------------------------------------------------------------------------- [ 20.06%]########################################------------------------------------------------------------ [ 40.39%]########################################------------------------------------------------------------ [ 40.39%]

In [22]:
config_meta_test = config_meta.compute()
config_meta_test['images'] = [i for i in config_meta_test['images'] if i['name'] in os.listdir(result_folder + '/' + site_p + '_' + processDate_p)]

#Task4: Download the files to folder ./{site}_{procossDate}/
download_res = t.download_files(pipeline_dict = config_meta_test,
                               site=site_p,
                               processDate=processDate_p,
                               result_folder=result_folder)

In [24]:
#config_meta_test

In [25]:
test = download_res.compute()

In [27]:
#Task5: Get the metadata for each file
workflow_meta = t.get_file_meta(pipeline_dict=test)

In [28]:
test2 = workflow_meta.compute()

In [32]:
#Task6: Write the metadata for each file to a human readable file (.json)
metadata_exported = t.write_pipeline_meta(pipeline_dict = test2['images'],
                                          site=site_p,
                                          processDate=processDate_p,
                                          result_folder = result_folder)

In [33]:
test3 = metadata_exported.compute()

In [11]:
test1 = h5_files.compute()
test = workflow_meta.compute()

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f1097c22700>, <methodcaller: map>, [<function download_file at 0x7f1093f13ca0>], (<class 'dict'>, [['pipeline_dict', {'images': [{'name': 'NEON_10_CPER_DP1_20170524_155746_reflectance.h5', 'size': 4369518341, 'md5': None, 'crc32': None, 'crc32c': 'ec2964c6', 'url': 'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D10/2017_CPER_3/L1/Spectrometer/ReflectanceH5/2017052414/NEON_10_CPER_DP1_20170524_155746_reflectance.h5'}, {'name': 'NEON_10_CPER_DP1_20170524_181919_reflectance.h5', 'size': 4261928967, 'md5': None, 'crc32': None, 'crc32c': '3b62174b', 'url': 'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D10/2017_CPER_3/L1/Spectrometer/ReflectanceH5/2017052414/NEON_10_CPER_DP1_20170524_181919_reflectance.h5'}, {'name': 'NEON_10_CPER_DP1_20170524_165356_reflectance.h5', 'size': 4819355943, 'md5': None, 'crc32': None, 'crc32c': 'bdf9ea66', 'url': 'https://st

AttributeError: 'function' object has no attribute 'map'

In [9]:
#pipeline_dict = {}
#pipeline_dict['images'] = test1
#pipeline_dict
workflow_meta.compute()

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f5c56ad7700>, <function download_file at 0x7f5c52f1bdc0>, [], (<class 'dict'>, [['pipeline_dict', {'images': [{'name': 'NEON_10_CPER_DP1_20170524_155746_reflectance.h5', 'size': 4369518341, 'md5': None, 'crc32': None, 'crc32c': 'ec2964c6', 'url': 'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D10/2017_CPER_3/L1/Spectrometer/ReflectanceH5/2017052414/NEON_10_CPER_DP1_20170524_155746_reflectance.h5'}, {'name': 'NEON_10_CPER_DP1_20170524_181919_reflectance.h5', 'size': 4261928967, 'md5': None, 'crc32': None, 'crc32c': '3b62174b', 'url': 'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D10/2017_CPER_3/L1/Spectrometer/ReflectanceH5/2017052414/NEON_10_CPER_DP1_20170524_181919_reflectance.h5'}, {'name': 'NEON_10_CPER_DP1_20170524_165356_reflectance.h5', 'size': 4819355943, 'md5': None, 'crc32': None, 'crc32c': 'bdf9ea66', 'url': 'https://storage.googleapis.co

KeyError: 'url'

## Register FLow and Start of Local Agent

In [ ]:
# Register the flow under the "tutorial" project
flow.register(project_name="Neon_AOP_BRDF_Mosaic")

In [ ]:
!prefect agent local start